# Group 3 Term Project 1
- Vibhu Bhardwaj (261113187)
- Yihui Chen (261103068)
- Hong Fei Jin (260927239)
- Utkarsh Pal (261121772)
- Haoying Xu (261109413)

## Import the data

In [11]:
# Import libraries
import pandas as pd
import numpy as np
import time

In [12]:
# Make it so that all columns will be visible
pd.set_option('display.max_columns', None)

In [13]:
# Import the data
st = time.time() # Get start time

df_business = pd.read_csv(r'C:\Users\jinho\Downloads\Mining\Projects\Team1\data\yelp_business.csv')
df_attributes = pd.read_csv(r'C:\Users\jinho\Downloads\Mining\Projects\Team1\data\yelp_business_attributes.csv')
df_hours = pd.read_csv(r'C:\Users\jinho\Downloads\Mining\Projects\Team1\data\yelp_business_hours.csv')
df_checkin = pd.read_csv(r'C:\Users\jinho\Downloads\Mining\Projects\Team1\data\yelp_checkin.csv')
df_review = pd.read_csv(r'C:\Users\jinho\Downloads\Mining\Projects\Team1\data\yelp_review.csv')
df_tip = pd.read_csv(r'C:\Users\jinho\Downloads\Mining\Projects\Team1\data\yelp_tip.csv')
df_user = pd.read_csv(r'C:\Users\jinho\Downloads\Mining\Projects\Team1\data\yelp_user.csv')

et = time.time() # Get end time
elapsed_time = et - st # Calculate execution time
print('Execution time:', round(elapsed_time/60, 2), 'minutes')

Execution time: 1.15 minutes


## Data pre-processing

In [14]:
# Remove the all rows where there are no business_id
df_business = df_business[df_business['business_id'].notna()]
df_attributes = df_attributes[df_attributes['business_id'].notna()]
df_hours = df_hours[df_hours['business_id'].notna()]
df_checkin = df_checkin[df_checkin['business_id'].notna()]
df_review = df_review[df_review['business_id'].notna()]
df_tip = df_tip[df_tip['business_id'].notna()]

# Replace all 'Na' and 'None' with NULL
df_business = df_business.replace({'Na': np.nan}, regex = True)
df_attributes = df_attributes.replace({'Na': np.nan}, regex = True)

In [15]:
# Get the number of stars (our target variable) of each business
df_rating = df_business[['business_id', 'stars', 'categories']]

# Get businesses that are restaurants
df_restaurants = df_rating[df_rating['categories'].str.contains('Restaurants|Food') == True]

In [16]:
# Breaking down df_attributes into smaller DataFrame concerning restaurants
df_dietary = df_attributes[['business_id', 'DietaryRestrictions_dairy-free', 'DietaryRestrictions_gluten-free',
                            'DietaryRestrictions_vegan', 'DietaryRestrictions_kosher',
                            'DietaryRestrictions_halal', 'DietaryRestrictions_soy-free',
                            'DietaryRestrictions_vegetarian']]
df_ambience = df_attributes[['business_id', 'Ambience_romantic', 'Ambience_intimate', 'Ambience_classy',
                            'Ambience_hipster', 'Ambience_divey', 'Ambience_touristy',
                            'Ambience_trendy', 'Ambience_upscale', 'Ambience_casual']]
df_parking = df_attributes[['business_id', 'BusinessParking_garage', 'BusinessParking_street',
                            'BusinessParking_validated', 'BusinessParking_lot',
                            'BusinessParking_valet', 'BikeParking']]
df_good = df_attributes[['business_id', 'GoodForMeal_dessert', 'GoodForMeal_latenight', 'GoodForMeal_lunch',
                         'GoodForMeal_dinner', 'GoodForMeal_breakfast', 'GoodForMeal_brunch']]
df_other = df_attributes[['business_id', 'ByAppointmentOnly', 'BusinessAcceptsCreditCards', 'RestaurantsPriceRange2',
                          'GoodForKids', 'WheelchairAccessible', 'Alcohol',
                          'HasTV', 'NoiseLevel', 'RestaurantsAttire',
                          'RestaurantsGoodForGroups', 'WiFi', 'RestaurantsReservations',
                          'RestaurantsTakeOut', 'RestaurantsTableService', 'OutdoorSeating',
                          'RestaurantsDelivery', 'Smoking', 'DriveThru',
                          'DogsAllowed', 'RestaurantsCounterService']]

## Recursive Feature Elimination on df_other

In [18]:
feat_other = pd.merge(df_restaurants, df_other, on = 'business_id', how = 'inner')
pd.set_option('display.max_columns', None)

We can do some further cleaning of the dataset:
- The entire RestaurantsPriceRange2 column is null so we can drop it
- We should also drop HasTV because the categories are "full_bar" and "beer_and_wine"
- We should drop RestaurantsGoodForGroups because the entire column is either 'True' or NULL
- Finally there is only one row in RestaurantsCounterService that is NOT NULL

In [19]:
feat_other = feat_other.drop(columns = ['RestaurantsPriceRange2', 'HasTV', 'RestaurantsGoodForGroups', 'RestaurantsCounterService', 'RestaurantsAttire', 'RestaurantsReservations'])

In [20]:
# Dummify remaining columns
feat_other = feat_other.replace({np.NaN : 0, 'False' : 0, 'True' : 1}, regex = True) 
feat_other['DriveThru'] = feat_other['DriveThru'].replace({np.NaN : 0, 'outdoor' : 1, 'yes' : 1, 'no': 0}, regex = True)

In [21]:
from sklearn import preprocessing
from sklearn import utils

X = feat_other.iloc[:, 4:]
y = feat_other['stars']

lab = preprocessing.LabelEncoder()
y_transformed = lab.fit_transform(y)

np.unique(y_transformed)

array([0, 1, 2, 3, 4, 5, 6, 7, 8], dtype=int64)

In [22]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(max_iter = 5000)
rfe = RFE(lr, n_features_to_select = 1)
model1 = rfe.fit(X,y_transformed)

In [23]:
pd.DataFrame(list(zip(X.columns,model1.ranking_)), columns = ['predictor','ranking']).sort_values(by = ['ranking'])

,predictor,ranking
12,DogsAllowed,1
5,WiFi,2
6,RestaurantsTakeOut,3
10,Smoking,4
4,NoiseLevel,5
9,RestaurantsDelivery,6
11,DriveThru,7
2,WheelchairAccessible,8
0,BusinessAcceptsCreditCards,9
8,OutdoorSeating,10


From here we can see that DogsAllowed, WiFi, RestaurantsTakeOut are good predictors. We can thus take these results to build our Linear Regression model and get more detailed results concerning accuracy and such. We can also repeat this for the other DataFrames created like Dietary, Ambience, Parking to get more detailed feedback and improve our linear regression model.

## Random Forest Classifier

We can start again as to not have the effects of the section above to effect our results here

In [27]:
# Import libraries
import pandas as pd
import numpy as np

# Import the data
business = pd.read_csv(r'C:\Users\jinho\Downloads\Mining\Projects\Team1\data\yelp_business.csv')
business_attributes = pd.read_csv(r'C:\Users\jinho\Downloads\Mining\Projects\Team1\data\yelp_business_attributes.csv')

In [29]:
# Get the number of stars (our target variable) of each business
df_rating = business[['business_id', 'stars', 'categories']]

# Take only the restaurants
restaurant = df_rating[df_rating['categories'].str.contains("restaurant|food")==True]
restaurant_2 = pd.merge(restaurant,business_attributes,on='business_id')

In [30]:
# Dumify variables
restaurant_2=restaurant_2.replace(to_replace={'Na':0,'True':1,'False':0,'beer_and_wine':1,
                                              'full_bar':1,'no':0,'none':0,'outdoor':1,'yes':1})

In [31]:
# Transform target into categorical data
star_df=restaurant_2['stars']
star_label = []
for star in star_df: #assign the percentile category to the new column
    if star <= 1:
        star_label.append('Very Poor')
    elif star > 1 and star <= 2:
        star_label.append('Poor')
    elif star > 2 and star <= 3:
        star_label.append('Average')
    elif star > 3 and star <= 4:
        star_label.append('Good')
    elif star > 4:
        star_label.append('Very Good')

In [32]:
#combine the new column to the df
restaurant_3 = pd.concat([restaurant_2, pd.DataFrame(star_label, columns=['star_label'])], axis=1)
restaurant_3['stars']=restaurant_3['stars'].astype('category') #make sure identified as categorical

#1. Parking related
X1 = restaurant_3[['BusinessParking_garage','BusinessParking_street',
                 'BusinessParking_lot','BusinessParking_valet','BusinessParking_validated','BikeParking']]

#2. Meal attributes
X2 = restaurant_3[['GoodForMeal_breakfast','GoodForMeal_brunch','GoodForMeal_lunch','GoodForMeal_dinner','GoodForMeal_dessert','GoodForMeal_latenight']]

#3. Ambiance
X3 = restaurant_3[['Ambience_romantic','Ambience_intimate','Ambience_classy','Ambience_hipster',
                   'Ambience_divey','Ambience_touristy','Ambience_trendy','Ambience_upscale','Ambience_casual']]

#4. Dietary Restrictions
X4 = restaurant_3[['DietaryRestrictions_kosher','DietaryRestrictions_gluten-free','DietaryRestrictions_vegan']]

#5. Other attrbutes
X5 = restaurant_3[['DogsAllowed','WiFi','RestaurantsTakeOut','DriveThru','OutdoorSeating','BusinessAcceptsCreditCards','ByAppointmentOnly','RestaurantsTableService',
                   'Smoking','NoiseLevel','RestaurantsDelivery','HasTV','WheelchairAccessible','GoodForKids','Alcohol']]

y = restaurant_3['star_label']

In [33]:
from sklearn.ensemble import RandomForestClassifier #random forest doing featyre selection
rf1=RandomForestClassifier(random_state=(5)) 

In [34]:
#1. Parking related
model1=rf1.fit(X1,y)
model1.feature_importances_
pd.DataFrame(list(zip(X1.columns,model1.feature_importances_)), columns = ['predictor','feature importance']).sort_values(by='feature importance',ascending=False)

,predictor,feature importance
5,BikeParking,0.222248
4,BusinessParking_validated,0.196237
1,BusinessParking_street,0.190517
3,BusinessParking_valet,0.171438
2,BusinessParking_lot,0.158058
0,BusinessParking_garage,0.061501


In [35]:
#2. Meal attributes
rf2=RandomForestClassifier(random_state=(5)) 
model2=rf2.fit(X2,y)
model2.feature_importances_
pd.DataFrame(list(zip(X2.columns,model2.feature_importances_)), columns = ['predictor','feature importance']).sort_values(by='feature importance',ascending=False)

,predictor,feature importance
3,GoodForMeal_dinner,0.302619
0,GoodForMeal_breakfast,0.276815
4,GoodForMeal_dessert,0.151012
1,GoodForMeal_brunch,0.103325
5,GoodForMeal_latenight,0.098074
2,GoodForMeal_lunch,0.068155


In [36]:
#3. Ambiance
#all in 0, should avoid this analysis
rf3=RandomForestClassifier(random_state=(5)) 
model3=rf3.fit(X3,y)
model3.feature_importances_
pd.DataFrame(list(zip(X3.columns,model3.feature_importances_)), columns = ['predictor','feature importance']).sort_values(by='feature importance',ascending=False)

,predictor,feature importance
0,Ambience_romantic,0.0
1,Ambience_intimate,0.0
2,Ambience_classy,0.0
3,Ambience_hipster,0.0
4,Ambience_divey,0.0
5,Ambience_touristy,0.0
6,Ambience_trendy,0.0
7,Ambience_upscale,0.0
8,Ambience_casual,0.0


In [37]:
#4. Dietary Restrictions
#all in 0, should avoid this analysis
rf4=RandomForestClassifier(random_state=(5)) 
model4=rf4.fit(X3,y)
model4.feature_importances_
pd.DataFrame(list(zip(X4.columns,model4.feature_importances_)), columns = ['predictor','feature importance']).sort_values(by='feature importance',ascending=False)

,predictor,feature importance
0,DietaryRestrictions_kosher,0.0
1,DietaryRestrictions_gluten-free,0.0
2,DietaryRestrictions_vegan,0.0


In [38]:
#5. Other attrbutes
rf5=RandomForestClassifier(random_state=(5)) 
model5=rf4.fit(X5,y)
model5.feature_importances_
pd.DataFrame(list(zip(X5.columns,model5.feature_importances_)), columns = ['predictor','feature importance']).sort_values(by='feature importance',ascending=False)

,predictor,feature importance
14,Alcohol,0.242065
0,DogsAllowed,0.205084
13,GoodForKids,0.142774
12,WheelchairAccessible,0.083351
3,DriveThru,0.078510
4,OutdoorSeating,0.054097
1,WiFi,0.040710
2,RestaurantsTakeOut,0.035527
9,NoiseLevel,0.030158
11,HasTV,0.026648


The results of the Random Forest will also help in identifying important predictors that the Yelp consultatnt may want to examine.